# 🔧 What is a Hyperparameter?
Hyperparameters are settings you define before training a model, such as:

- Number of layers
- Number of neurons per layer
- Learning rate
- Batch size
- Activation functions
- Optimizer

These are not learned by the model — you must manually select or automatically search for them.

# 🚀 What is Keras Tuner?
Keras Tuner helps you automatically find the best combination of hyperparameters using smart search algorithms.

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

# 📥 Load and normalize MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

# 🛠️ Build model with all tunable components
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))

    # 🔁 Tune number of hidden layers (1–3)
    for i in range(hp.Int("num_layers", 1, 3)):
        # Dense layer with tunable units
        model.add(layers.Dense(
            units=hp.Int(f"units_{i}", min_value=64, max_value=512, step=64),
            activation=hp.Choice(f"activation_{i}", ["relu", "tanh", "sigmoid"])
        ))

        # Optional Batch Normalization
        if hp.Boolean(f"batch_norm_{i}"):
            model.add(layers.BatchNormalization())

        # Optional Dropout
        model.add(layers.Dropout(
            rate=hp.Float(f"dropout_{i}", 0.0, 0.5, step=0.1)
        ))

    # Output layer
    model.add(layers.Dense(10, activation="softmax"))

    # 🔧 Tune optimizer & learning rate
    optimizer = hp.Choice("optimizer", ["adam", "rmsprop", "sgd"])
    learning_rate = hp.Float("learning_rate", 1e-4, 1e-2, sampling="log")

    # Compile the model
    if optimizer == "adam":
        opt = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == "rmsprop":
        opt = keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        opt = keras.optimizers.SGD(learning_rate=learning_rate)

    model.compile(
        optimizer=opt,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# 🔍 Set up tuner
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=1,
    directory="my_tuner_dir",
    project_name="mnist_full_tuning"
)

# 🚀 Start search
tuner.search(
    x_train, y_train,
    epochs=5,
    validation_split=0.2,
    batch_size=64,
    verbose=1  # to print Trial summary
)

# 📋 Get the best trial hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("\n✅ Best Hyperparameters:")
for param in best_hps.values:
    print(f"{param}: {best_hps.get(param)}")


Trial 5 Complete [00h 00m 28s]
val_accuracy: 0.9268333315849304

Best val_accuracy So Far: 0.9739166498184204
Total elapsed time: 00h 02m 31s

✅ Best Hyperparameters:
num_layers: 1
units_0: 192
activation_0: relu
batch_norm_0: True
dropout_0: 0.30000000000000004
optimizer: rmsprop
learning_rate: 0.002139602158318416
units_1: 64
activation_1: tanh
batch_norm_1: False
dropout_1: 0.0
units_2: 128
activation_2: sigmoid
batch_norm_2: True
dropout_2: 0.0
